In [ ]:
from pnet.core import core
from pnet.qa import analysis
from datetime import datetime, date 

import duckdb as dd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

_util = core.Util()
_analysis = analysis.Analysis()

In [ ]:
# the dataset return is enrich with other additional fields
df = _analysis.get_enrich_dataset(False)


### Fraud trasaction (consumer & merchant GPS) 

##### Plot consumer and merchant transaction with dotted line
###### - The map shpw that relatinship between consumer & merchant is predictation. For eg, no oversea transaction or purchase located far away from home
###### - Detected invalid merchant location, as some located on the sea and top of mountain.

In [ ]:
#_analysis.plot_map_for_fraud_transaction(df)


###### Total number of FRAUD transaction and amount (group by card types)

In [ ]:
_sql = "SELECT is_fraud, card_type, count(*) num_of_transaction, sum(amt) sum_amount from {df} WHERE is_fraud = '1' GROUP BY 1,2 ORDER BY 2 DESC"
_util.spark.sql(_sql, df=df).show()

###### Total number of FRAUD transaction and amount (group by state and city)

In [ ]:
_sql = "SELECT is_fraud, state, count(*) num_of_transaction, sum(amt) sum_amount from {df} WHERE is_fraud = '1' GROUP BY 1,2 ORDER BY 3 DESC"
_util.spark.sql(_sql, df=df).show(df.count(),False)

In [ ]:
# Destroy object
df.unpersist()
df = None

##### Perform data transformation for chart plotting

In [ ]:
# load dataset as Pandas
_sql = "SELECT * FROM read_parquet ('/code/data/ingestion/final/part*.parquet') ORDER BY 1"
df_final = df_analysis = dd.sql(_sql).df()


# Configure format
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# function declaration
def age(dob): 
    born = datetime.strptime(dob, "%Y-%m-%d").date() 
    today = date.today() 
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day)) 


# Pandas transformation
df_analysis['trans_date'] = df_analysis['trans_date_trans_time'].str[:10]
df_analysis['trans_date'] = pd.to_datetime(df_analysis['trans_date'], errors='coerce')
df_analysis['day_of_week'] = df_analysis['trans_date'].dt.day_name()
df_analysis['trx_count'] = 1
df_analysis['age'] = df_analysis['dob'].apply(age) 


df_category = df_analysis[['category','trans_num']].groupby(['category']).count().reset_index()
df_category.columns = ['Category','category_count']
df_category['percent'] = (df_category['category_count']/df_category['category_count'].sum())*100

##### Chart show number of transaction (group by category)

In [ ]:
fig = px.pie(df_analysis, values='trx_count', names='category', title='Total number of transaction (by category)')
fig.show()


##### Chart show number of transaction (group by day of week)

In [ ]:
fig = px.pie(df_analysis, values='trx_count', names='day_of_week', title='Total number of transaction (by day of week)')
fig.show()

In [ ]:
fig = px.pie(df_analysis, values='trx_count', names='gender', title='Total number of transaction (by gender)')
fig.show()

##### Age population

In [ ]:
df_analysis['age'].describe()

In [ ]:
df_fraud = df_analysis[['category','is_fraud','trans_num']].groupby(['category','is_fraud']).count().reset_index()
df_fraud.columns = ['Category','is_fraud','count']

df_fraud = df_fraud.merge(df_category[['Category','category_count','percent']],how='inner',\
                                  left_on='Category',right_on='Category')


df_fraud['percent_grp'] = (df_fraud['count']/df_fraud['category_count'])*100
df_fraud.sort_values(by = ['category_count'], ascending=False)

#### Fraud Analysis
###### - For fraud transaction, NY have the highest transaction SUM of value (295548.64)
###### - VISA is the primary target of fraudster with total number of transaction 2441 (amount 1319668.70)


#### Overall Observation (Overall)
###### - Consumer spending were very much focus on gas (10.2%) and grocery (9.54%)
###### - Based on the chart, consumer mostly spending on Sunday and Monday
###### - Male contribute the highest number of transaction
###### - The average aging population is between 38 and 62 years old
